In [1]:
import numpy as np
import skimage
import skimage.data

import ipywidgets as widgets
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, Layout, HBox
from skimage.transform import rescale, warp, ProjectiveTransform

import matplotlib.transforms
import matplotlib

# Q & A

## Questions

1. As mentioned in the first section of the paper, just curious what causes spatial transforms to apply "attention" to relevant sections of an image? Is it one of: "the localisation network, grid generator, and sampler" or the combination of all 3 of them that leads to attentiveness?

2. Why is Equation (2) of the paper the "attention" transform? Equation (2) being the matrix:

$$\begin{bmatrix}
s & 0 & t[x] \\
0 & s & t[y] \\
\end{bmatrix}$$

3. Following up on Section 3.4, how/why do spatial transformers minimize the overall cost function of their parent CNN during training?

4. Continuing with Section 3.4 again, why do spatial transformers limit the number of objects that can be modeled by a feed-forward network?

5. The conclusion section says spatial transforms learn without making changes to their parent CNN's cost function. But Section 3.4 (please see Question #3 above) seems to suggest that spatial transformers do indeed make changes to their parent CNN's cost function. So I'm confused. Please advise if I'm mixing up concepts here and may be the authors are meaning something else in Section 3.4's cost minimization discussion, and something entirely different in the Conclusion section.

6. This seems really useful and cool. Do people know what orientation the object is? How do we know that we have to put it face up or face down? 


7. Can this be extended beyond just normalizing data to creating new data? Or to learn like a manifold?


## Responses

##### 1. As mentioned in the first section of the paper, just curious what causes spatial transforms to apply "attention" to relevant sections of an image? Is it one of: "the localisation network, grid generator, and sampler" or the combination of all 3 of them that leads to attentiveness?

it is of course, a combination of all 3, but let's consider them in the order of backpropagation.

1. sampler

We have your image U and your spatially transformed image V.  Interpolation will determine what sets of points $u_{ij}: {i,j}\in HW$ 
the gradients which pass through, and what contributions they have.

2. grid generator

This piece is really the definition of a spatial transformation. It is how $\theta$ relates U and V in terms of mapping a point $v_{ij}$ to a position.  Then the sampler determines which $u_{ij}$ to associate for that position.  Now, that means that the grid generator is really what ties $\theta$ to $U$ and $V$

3. localizer network

Gradients of $\theta$ will propogate through the localization network, allowing it to tune the parameters $\theta$


##### 2. Why is Equation (2) of the paper the "attention" transform? Equation (2) being the matrix:

$$\begin{bmatrix}
s & 0 & t[x] \\
0 & s & t[y] \\
\end{bmatrix}$$

In [2]:
N = 128
image = np.zeros((N, N))
image[0:8, :] = 1
image[:, 0:8] = 1
image[-8:, :] = 1
image[:, -8:] = 1
def f(tx, ty, s,):
    fig=plt.figure(figsize=(6, 6), dpi= 80, facecolor='w', edgecolor='k')
    mat = np.array([
        [s ,0, -tx],
        [0, s, -ty],
        [0, 0, 1]
    ])
    xdim = image.shape[1]
    ydim = image.shape[0]
    
    img = warp(image, mat, output_shape=image.shape, 
               order=3, mode='constant')
    plt.imshow(img, cmap='gray')
    transform = matplotlib.transforms.Affine2D(matrix=mat)
    plt.grid(transform=transform, visible=True)

def reset_values(b):
    for child in plot2.children:
        if not hasattr(child, 'description'):
            continue
        elif child.description in ['tx', 'ty', 'θ', 'kx', 'ky']:
            child.value = 0
        elif child.description in ['s']:
            child.value = 1.0

reset_button = widgets.Button(description = "Reset")
reset_button.on_click(reset_values)

x2 = widgets.IntSlider(min=0, max=800, step=10, orientation='vertical', description='$t_x$')
y2 = widgets.IntSlider(min=0, max=800, step=10, orientation='vertical', description='$t_y$')
s = widgets.FloatSlider(min=1.0, max=8.0, value=1, orientation='vertical', description='$s$')
plot = interactive(f, tx=x2, ty=y2, s=s)
layout = Layout(display='flex', flex_flow='row')

In [3]:
plot.update()
display(HBox([plot.children[-1]]))
display(HBox([*plot.children[:-1]], layout=layout))

##### 3. Following up on Section 3.4, how/why do spatial transformers minimize the overall cost function of their parent CNN during training?

As I could understand it, the spatial transformer network will tend to find an transform which enhances the location of activations. It will learn somehow that activations were actually translocated from where they will be maximized, although it's not at all clear how good or certain the convergence of this process is to me.

##### 4. Continuing with Section 3.4 again, why do spatial transformers limit the number of objects that can be modeled by a feed-forward network?

In general, I think they are stating that, if you were trying to do some bbox network, the problem with utilizing spatial transformers is that you would have to "fix" the architecture to support a set number of transformers.  I somewhat disagree with this, in that the spatial transformers may themselves be able to classify multiple objects.  

The other alternative may be a conception of a RNN-spatial transformer, which could feed itself results, trying to predict if there is one or many objects within, then masking out objects for which it finds a satisfactory bounding box, and feeding that input to itself again?  Or some other recursive implementation where spatial transformers attempt to cluster objects recursively, but this may be some unworkable tree-structure.

##### 5. The conclusion section says spatial transforms learn without making changes to their parent CNN's cost function. But Section 3.4 (please see Question #3 above) seems to suggest that spatial transformers do indeed make changes to their parent CNN's cost function. So I'm confused. Please advise if I'm mixing up concepts here and may be the authors are meaning something else in Section 3.4's cost minimization discussion, and something entirely different in the Conclusion section.

Yes, in the first statement I believe they are simply saying there is no need to add in some spatial transformation driven loss.  One could imagine the need to impose loss on the parameters $\theta$, but it works without it.  On the other hand, I don't think that their work has really made any comment on such a loss function being added, it's possible that it would be a good idea even if it's unnecessary.

##### 6. This seems really useful and cool. Do people know what orientation the object is? How do we know that we have to put it face up or face down? 

We can feed the "pose" (i.e., $\theta$) into the later parts of the network. As for face up or face down, I believe this comes down to a supervised approach. In the discussion I found online, the authors admit that they essentially had to force the localization network to "initialize" at an identity matrix. They don't really discuss this method at all, as it seems somewhat difficult to me for an arbitrary localization network to force the output parameters.  Probably they just initialize extremely small weights?

##### 7. How does the spatial transformer learn capturing the region of target object? I have a hard time mapping the math of spatial transformer to the output it generates.

Actually, they don't really talk about this. The spatial transformation is certainly "differentiable", so in that regard it can learn the parameters $\theta$. The values $x_i^s$, $y_i^s$ are determined by the transform parameters $\theta$. Those gradients, $\frac{\partial V} {\partial U}$, $\frac{\partial V} {\partial x}$, $\frac{\partial x} {\partial \theta}$ seem to be enough to give it what direction it could shift to do better. My guess would be that batch size plays in important role in stabilizing how the network learns a better spatial transformation in a stable manner.

The most difficult part about imagining this in general, is that if the network is training both the transformer and the rest of the network at once, it feels like the results could be unstable. If object pose is mostly uniform with edge cases that throw off the network, train the network for some time to recognize basic cases, then plug in a spatial transformer. It will learn to shift the bad cases to places where they experience larger activations.


##### 8. Can this be extended beyond just normalizing data to creating new data? Or to learn like a manifold?

I have some ideas...

I think so, although I'm not exactly sure how to extract the manifold for these simple supervised cases. There may be another little revolution in visualization techniques which would have to come first, similar to how we visualize higher level features for CNNs, how do we visualize how features drive rotation/translation/scale/skew or even thin-plate spline distortions?  That could be a fantastic project all its own.

Another consideration might be variational autoencoders. For example, allowing the input image to be invertably deformed prior to encoding, then storing the deformation parameters in the latent space might put less bearing on the network to "locate" features before storing them. I actually think this lends itself more to medical imaging than regular photographs, due to obscuration of objects.

As a more wild proposal, I think something like using spatial transformer networks to isolate appendages, then feeding transformation parameters into the model (to keep track of pose as well as relative spatial locations) could result in some sort of generative model for dynamic poses of humans/animals. This would require some very clever engineering I think, to ensure that the model knows the implicit spatial relationships of the appendages.

Finally, what was really not explored in the paper is the concept of adding energy constraints to the transformer model. By putting some physical limitations on the bending of the model, you might be able to marry the spatial transformations with more advanced contour methods or modelling of fluids, materials.
